# Multi-omics analisis with LRA cluster and COCA

* [Load data and preprocessing](#Load-data-and-preprocessing)
* [Dataset alignment](#Dataset-alignment)
* [LRA cluster](#LRA-cluster)
    * [LRA cluster R implementation](#LRA-cluster-R-implementation)
    * [LRA cluster python implementation](#LRA-cluster-python-implementation)
* [Clinic Data](#Clinic-Data)
* [COCA](#COCA)




In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import rpy2.robjects as ro
from rpy2.robjects import r
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
import pdb
from tqdm import tqdm
from rpy2.robjects.conversion import localconverter
from myclass.CleanMergeDataset import Clean_Merge_Dataset
from myclass.BonferroniTtest import Bonferroni_Ttest
#import plotly.express as px
from sklearn.cluster import DBSCAN, KMeans, AgglomerativeClustering, SpectralClustering
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

#reload(myclass.BonferroniTtest)
#reload(myclass.CleanMergeDataset)
from rpy2.robjects.packages import STAP
import rpy2.robjects as robjects

## Load data and preprocessing

Loading the datasets, clean them through class *Clean_Merge_Dataset*, reduce the number of features with *Bonferroni_Ttest*.

In [ ]:
import json

data_normal = pd.read_pickle('./SimilarityNetworkFusion/data-ready/miRNA_dataframe_normal').replace('/', '\\')
data_tumor = pd.read_pickle('./SimilarityNetworkFusion/data-ready/miRNA_dataframe').replace('/', '\\')
df_miRNA, y_miRNA = Clean_Merge_Dataset(name='miRNA').transform(data_normal, data_tumor)

data_normal = pd.read_pickle('./SimilarityNetworkFusion/data-ready/RNA_dataframe_normal').replace('/', '\\')
data_tumor = pd.read_pickle('./SimilarityNetworkFusion/data-ready/RNA_dataframe').replace('/', '\\')
df_RNA, y_RNA = Clean_Merge_Dataset(name='RNA').transform(data_normal, data_tumor)


data_normal = pd.read_pickle('./SimilarityNetworkFusion/data-ready/illumina-27-450-normal').replace('/', '\\')
data_tumor = pd.read_pickle('./SimilarityNetworkFusion/data-ready/illumina450-27-tumor').replace('/', '\\')
df_illumina, y_illumina= Clean_Merge_Dataset(name='illumina').transform(data_normal, data_tumor)

dataset_RNA = Bonferroni_Ttest(label_case_id_into_X=True, alpha=0.05).fit_transform(df_RNA, y_RNA)
dataset_miRNA_m = Bonferroni_Ttest(label_case_id_into_X=True, alpha=0.05).fit_transform(df_miRNA, y_miRNA)
dataset_illumina = Bonferroni_Ttest(label_case_id_into_X=True, alpha=0.05).fit_transform(df_illumina, y_illumina)



    

   
    

    



# Data aligment

In this section we eliminate all the examples that are not present in all 3 omics. We noticed that there are samples with the same case id but with different labels. We decided to use them for our analysis.

In [ ]:
def uniqueCaseID(data):
    new = {}
    new["case_id1"] = []
    data = data.copy()

    data  = data[data["label"] != "CPTAC-3"]
    
    data = data.sort_values(by=['case_id']).reset_index(drop=True)
    for i,row in data.iterrows():
        new["case_id1"].append(data.loc[i,"case_id"]+"_"+str(data.loc[i,'label']))
   
    new_df = pd.DataFrame(new, columns = ["case_id1"])
    conc_df = pd.concat([new_df,data], axis=1)
    conc_df = conc_df.drop(["case_id"], axis=1)
    
    return conc_df



illu = uniqueCaseID(dataset_illumina[0])
rna = uniqueCaseID(dataset_RNA[0])
mirna = uniqueCaseID(dataset_miRNA_m[0])





i1 = set(illu['case_id1'])
i2 = set(mirna['case_id1'])
i3 = set(rna['case_id1'])

#Prendiamo i case_id_label che stanno in tutti e 3 le omiche
distinct_case_id = [x for x in i2 if x in i1 and x in i3]


#print(distinct_case_id)
#Filtriamo i case_id_label che non stanno in tutte e 3 le omiche
illu = illu.loc[illu['case_id1'].isin(distinct_case_id)]
rna = rna.loc[rna['case_id1'].isin(distinct_case_id)]
mirna = mirna.loc[mirna['case_id1'].isin(distinct_case_id)]

illu = illu.sort_values(by=['case_id1']).reset_index(drop=True)
rna = rna.sort_values(by=['case_id1']).reset_index(drop=True)
mirna = mirna.sort_values(by=['case_id1']).reset_index(drop=True)




label = illu['label']
label[label == False] ='False'

# LRA cluster

In the following two sections we analyze the LRA cluster algorithm, an example of early multi-omics integration. We have re-implemented this algorithm in python, trying to improve its efficiency by eliminating some superfluous operations or re-implementing them using more efficient python libraries such as jax.LRA clsuter extract features that can be usefull for a clustering algorithm.We tested it with the **Agglomerative clustering, Spectral clustering and KMeans algorithms**.

## LRA cluster R implementation

In this section we run the official R implementation of LRA cluster.We run the R code by using the rpy2 library.

In [ ]:
with localconverter(ro.default_converter + pandas2ri.converter):
  r_illu_d = ro.conversion.py2rpy(illu.drop(["case_id1","label"], axis=1))

with localconverter(ro.default_converter + pandas2ri.converter):
  r_miRNA_d = ro.conversion.py2rpy(mirna.drop(["case_id1","label"], axis=1))

with localconverter(ro.default_converter + pandas2ri.converter):
  r_rna_d = ro.conversion.py2rpy(rna.drop(["case_id1","label"], axis=1))

import rpy2.robjects as robjects
from rpy2.robjects.packages import SignatureTranslatedAnonymousPackage


##LRA cluster R
file_LRA_cluster = open("LRAcluster/R/LRAcluster.R",'r')

string_LRA_cluster = ''.join(file_LRA_cluster.readlines())

LRA_cluster = SignatureTranslatedAnonymousPackage(string_LRA_cluster, "LRAcluster")

#Trasformiamo in matrici i 3 dataset
matrix_r_miRNA_d = LRA_cluster.getMatrix(r_miRNA_d)
matrix_r_illu_d = LRA_cluster.getMatrix(r_illu_d)
matrix_r_rna_d = LRA_cluster.getMatrix(r_rna_d)

#Uniamo le matrici --> data
import time;

unionMatrix = LRA_cluster.unionMatrix3(matrix_r_illu_d, matrix_r_rna_d, matrix_r_miRNA_d)
start_time = time.time()
output_R = LRA_cluster.LRAcluster(unionMatrix,3)
print(start_time - time.time())
with localconverter(ro.default_converter + pandas2ri.converter):
    output = ro.conversion.rpy2py(output_R[0])

clustering = KMeans(n_clusters=3, max_iter=500).fit(output)
clustering1 = AgglomerativeClustering(n_clusters=3, linkage='ward').fit(output)
clustering2 = SpectralClustering(n_clusters=3, assign_labels='discretize').fit(output)


print("KMEANS")
print("silhuette: ", silhouette_score(output, clustering.labels_))
print("rand indexa: ", adjusted_rand_score(label, clustering.labels_))
print("AGGLOMERATIVE CLUSTERING")
print("silhuette: ", silhouette_score(output, clustering1.labels_))
print("rand indexa: ", adjusted_rand_score(label, clustering1.labels_))
print("SPECTRAL CLUSTERING")
print("silhuette: ", silhouette_score(output, clustering2.labels_))
print("rand indexa: ", adjusted_rand_score(label, clustering2.labels_))

## LRA cluster python implementation

In this section we perform our python implementation of the LRA cluster algorithm.

In [ ]:
#LRA cluster python implementation
label = illu['label']
label[label == False] ='False'

from PyLRAcluster.LRACluster import LRAcluster
import time


illu_m = np.transpose(illu.drop(["case_id1","label"], axis=1).to_numpy())
rna_m = np.transpose(rna.drop(["case_id1","label"], axis=1).to_numpy())
mirna_m = np.transpose(mirna.drop(["case_id1","label"], axis=1).to_numpy())

start_time = time.time()
output,p = LRAcluster([illu_m,rna_m,mirna_m],['gaussian','poisson','poisson'],3)
print(start_time - time.time())

clustering = KMeans(n_clusters=3, max_iter=500).fit(output)
clustering1 = AgglomerativeClustering(n_clusters=3, linkage='ward').fit(output)
clustering2 = SpectralClustering(n_clusters=3, assign_labels='discretize').fit(output)


print("KMEANS")
print("silhuette: ", silhouette_score(output, clustering.labels_))
print("rand indexa: ", adjusted_rand_score(label, clustering.labels_))
print("AGGLOMERATIVE CLUSTERING")
print("silhuette: ", silhouette_score(output, clustering1.labels_))
print("rand indexa: ", adjusted_rand_score(label, clustering1.labels_))
print("SPECTRAL CLUSTERING")
print("silhuette: ", silhouette_score(output, clustering2.labels_))
print("rand indexa: ", adjusted_rand_score(label, clustering2.labels_))



In [ ]:
##Plot graph##
le = LabelEncoder()
labels_num = le.fit_transform(clustering2.labels_)
plt.figure('LRA 3 dimension Spectral clustering', figsize=(15,15))
ax = plt.axes(projection = '3d')
ax.legend()
ax.scatter(output[:,0],output[:,1],output[:,2],c=labels_num);

# Clinic Data

We want to apply the information retrieved from clinical-data to the results obtained by the best clustering  (Spectral clustering). The link between the datas is the case_id. The fields chosen from clinical-data's file are:
- tumor_stage: stage of tumor.
- prior_malignancy: precedent diagnosys of tumors.
- age_at_diagnosis: patient's age which is diagnosed the tumor. We have considered only the year's decades.
- morphology: kind of tumor, which is identified from a code id.
- cigarettes per day: the average number of cigarettes smoked by the patient in one day.

In [ ]:
# DATI CLINICI
import os
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

"""
    Class to extract the clinical_case from the json file.
    It returns a dataframe in the following form:
    case_id| ...[clinical case]...| label.
"""
#set the cluster predicted labels as the labels predicted by Spectral clustering
y_pred = clustering2.labels_

#Class used to extract clinical data from the json file.
class ExtractClinicalCase:
    def __init__(self, cases_id):
        with open('./SimilarityNetworkFusion/data-ready/clinical.cases_selection.2020-11-12.json', 'r') as json_file:
            data = json.load(json_file)

        remove_el = list()
        for el in data:
            if el['case_id'] not in cases_id:
                remove_el.append(el)

        for el in remove_el:
            data.remove(el)
        
        clinical_data = {'case_id': '',
                        'tumor_stage': '',
                        'prior_malignancy': '',
                        'age_at_diagnosis': None,
                        'morphology': '',
                        'label': ''}

        self.df = pd.DataFrame(data=[], columns=clinical_data.keys())

        for i, el in enumerate(data):
            clinical_data['case_id'] = el['case_id']
            clinical_data['tumor_stage'] = el['diagnoses'][0]['tumor_stage']
            clinical_data['prior_malignancy'] = el['diagnoses'][0]['prior_malignancy']
            if el['diagnoses'][0]['age_at_diagnosis'] is not None:
                value = int(el['diagnoses'][0]['age_at_diagnosis'])/365
                clinical_data['age_at_diagnosis'] = self.__truncate__(value)

            clinical_data['morphology'] = el['diagnoses'][0]['morphology']
            if el['exposures'][0]['cigarettes_per_day'] is not None:
                value = int(el['exposures'][0]['cigarettes_per_day'])
                clinical_data['cigarettes_per_day'] = value

            self.df = self.df.append(pd.DataFrame(clinical_data, index=[i]), ignore_index=True)
        
    def get_df_clinical_case(self):
        return self.df

    def __truncate__(self, n, decimals=-1):
        """
            Function to take the decade of the age.
        """
        multiplier = 10 ** decimals
        return int(n * multiplier) / multiplier
                



cases_id = [el.split('_')[0] for el in rna['case_id1']]
df_clinical_case = ExtractClinicalCase(cases_id).get_df_clinical_case()

df_clinical_case.sort_values(by='case_id', inplace=True)
df = pd.DataFrame()
df['case_id'] = cases_id
df['label'] = y_pred
clinica_cases_id = [el.replace('\n', '') for el in df_clinical_case['case_id']]

label = df.loc[df['case_id'].isin(clinica_cases_id)]['label']
df_clinical_case['label'] = label


#Function to plot clinical data 
def plot_clinical_data(df_clinical_case, name_clinical_field, title=''):
    dict_ = {}
    df = pd.DataFrame()
    df_clinical_case.dropna(inplace=True)


    for l in range(0, 3):
        dict_['Cluster'] = l
        for el in set(df_clinical_case[name_clinical_field]):
            el_cluster = df_clinical_case[df_clinical_case['label'] == l]
            count = el_cluster[el_cluster[name_clinical_field] == el]['case_id'].count()
            dict_[el] = count

        df = df.append(pd.DataFrame(dict_, index=[0]))

    fig = go.Figure()
    
    for key in df.columns[1:]:
        fig = fig.add_trace(
            go.Bar(name=key, x=df['Cluster'], y=df[key])
        )
    
    # Change the bar mode
    fig.update_layout(barmode='stack',
                      title=title,
                      xaxis_title="Clusters",
                      yaxis_title="Count")
    fig.show() 
    return

 
plot_clinical_data(df_clinical_case, 'prior_malignancy', 'Prior Malignancy Distribution')
plot_clinical_data(df_clinical_case, 'age_at_diagnosis', 'Age at diagnosis Distribution')
plot_clinical_data(df_clinical_case, 'tumor_stage', 'Tumor stage Distribution')
plot_clinical_data(df_clinical_case, 'morphology', 'Morphology Distribution')
plot_clinical_data(df_clinical_case, 'cigarettes_per_day', 'Cigarettes per day Distribution')

## COCA 

In this section we have implemented the cluster of clusters algorithm (COCA). This algorithm was performed on the best combinations of preprocessing and algorithms that emerged from the single omic analysis.

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

#Clustering a singola omica
def vectorIndicator(indicator_vector_matrix,labels):
    new_encodings = []
    for i in range(labels.shape[0]):
        label_encoding = [0,0,0]
        label_encoding[labels[i]] = 1 # 0 -> [1,0,0] 1 ->[0,1,0] 2->[0,0,1]
        new_encodings.append(label_encoding)
    if indicator_vector_matrix.shape[0] == 0:
        indicator_vector_matrix = np.array(new_encodings)
    else:
        indicator_vector_matrix = np.concatenate((indicator_vector_matrix,np.array(new_encodings)),axis=1)
    return indicator_vector_matrix

def compute_vector_indicator(*args):
    indicator_vector_matrix = np.array([])
    for a in args:
        indicator_vector_matrix = vectorIndicator(indicator_vector_matrix,a.labels_)
    return indicator_vector_matrix


illu_new = uniqueCaseID(df_illumina)
mirna_new =uniqueCaseID(df_miRNA)
rna_new = uniqueCaseID(df_RNA)

i1 = set(illu_new['case_id1'])
i2 = set(mirna_new['case_id1'])
i3 = set(rna_new['case_id1'])

#Prendiamo i case_id_label che stanno in tutti e 3 le omiche
distinct_case_id = [x for x in i2 if x in i1 and x in i3]


#print(distinct_case_id)
#Filtriamo i case_id_label che non stanno in tutte e 3 le omiche
illu_new = illu_new.loc[illu_new['case_id1'].isin(distinct_case_id)]
rna_new = rna_new.loc[rna_new['case_id1'].isin(distinct_case_id)]
mirna_new = mirna_new.loc[mirna_new['case_id1'].isin(distinct_case_id)]

illu_new = illu_new.sort_values(by=['case_id1']).reset_index(drop=True)
rna_new = rna_new.sort_values(by=['case_id1']).reset_index(drop=True)
mirna_new = mirna_new.sort_values(by=['case_id1']).reset_index(drop=True)



label = illu_new['label']
label[label == False] = 'False'

#No bonferroni
illu_new = illu_new.drop(['case_id1','label'],axis=1)
mirna_new = mirna_new.drop(['case_id1','label'],axis=1)
rna_new = rna_new.drop(['case_id1','label'],axis=1)

#With bonferroni
illu_r = illu.drop(['case_id1','label'],axis=1)
mirna_r = mirna.drop(['case_id1','label'],axis=1)
rna_r = rna.drop(['case_id1','label'],axis=1)

SEED = 9
#mirna->Bonferroni->Maxminscaler->PCA->Spectral cluestering
pipe = Pipeline([('min_max_scale', MinMaxScaler()),('pca', PCA(0.9)),('spectral_clustering',SpectralClustering(n_clusters=3, assign_labels='discretize'))])
c1 = pipe.fit(mirna_r)[-1]
#mirna->Bonferroni->Maxminscaler->PCA->Spectral cluestering
pipe = Pipeline([('min_max_scale', MinMaxScaler()),('agglomerative_clustering',AgglomerativeClustering(n_clusters=3, linkage='ward'))])
c2 = pipe.fit(mirna_r)[-1]

#rna
#rna->Standardscaler->PCA->Kmeans
pipe = Pipeline([('standard_scaler', StandardScaler()),('pca', PCA(0.9)),('kmeans_clustering',KMeans(n_clusters=3, max_iter=500,random_state=SEED))])
c3 = pipe.fit(rna_new)[-1]
#rna->Bonferroni->MaxminScaler->Kmeans
pipe = Pipeline([('min_max_scale', MinMaxScaler()),('kmeans_clustering',KMeans(n_clusters=3, max_iter=500,random_state=SEED))])
c4 = pipe.fit(rna_r)[-1]

#illumina
#Bonferroni->PCA->Spectral cluestering
pipe = Pipeline([('pca', PCA(0.9)),('spectral_clustering',SpectralClustering(n_clusters=3, assign_labels='discretize'))])
c5 = pipe.fit(illu_r)[-1]
#Bonferroni->PCA->Agglomerative
pipe = Pipeline([('pca', PCA(0.9)),('agglomerative_clustering',AgglomerativeClustering(n_clusters=3, linkage='ward'))])
c6 = pipe.fit(illu_r)[-1]




X = compute_vector_indicator(c1,c2,c3,c4,c5,c6)


# setting distance_threshold=0 ensures we compute the full tree.

model = AgglomerativeClustering(n_clusters=3)
model = model.fit(X)

print("silhuette: ", silhouette_score(X, model.labels_))
print("rand indexa: ", adjusted_rand_score(label, model.labels_))


In [ ]:
y_pred = model.labels_
df_clinical_case = ExtractClinicalCase(cases_id).get_df_clinical_case()

df_clinical_case.sort_values(by='case_id', inplace=True)
df = pd.DataFrame()
df['case_id'] = cases_id
df['label'] = y_pred
clinica_cases_id = [el.replace('\n', '') for el in df_clinical_case['case_id']]

label = df.loc[df['case_id'].isin(clinica_cases_id)]['label']
df_clinical_case['label'] = label

plot_clinical_data(df_clinical_case, 'prior_malignancy', 'Prior Malignancy Distribution')
plot_clinical_data(df_clinical_case, 'age_at_diagnosis', 'Age at diagnosis Distribution')
plot_clinical_data(df_clinical_case, 'tumor_stage', 'Tumor stage Distribution')
plot_clinical_data(df_clinical_case, 'morphology', 'Morphology Distribution')
plot_clinical_data(df_clinical_case, 'cigarettes_per_day', 'Cigarettes per day Distribution')